In [1]:
%store -r city_coord
%store -r air_coord
%store -r office_info
%store -r office_coords
%store -r of_size
%store -r of_ref

import mapping_functions as M
import ranking_functions as R
import location_functions as L

import os
import re
import pandas as pd
from dotenv import load_dotenv
from pymongo import MongoClient
from bs4 import BeautifulSoup
import geocoder
import requests
import folium
from folium.plugins import MousePosition
from folium.plugins import Draw
from geopy.distance import distance as getDistance
load_dotenv()


dbName = 'companies'

db,start = L.connectCollection(dbName,'startups')
db,old_comp = L.connectCollection(dbName,'old_comp')
db,starbucks = L.connectCollection(dbName,'starbucks')
db,schools = L.connectCollection(dbName,'schools')
db,night = L.connectCollection(dbName,'night')

In [2]:
names = ["Old companies","Starbucks","Night bars","Schools","Startups","Airport","Offices"]  
collections = [old_comp,starbucks,night,schools,start]
colors = ["red","darkgreen","orange","lightblue","green"]

In [3]:
M.mapResults(city_coord,15,names,colors,collections,air_coord,office_coords)

In [4]:
weight = []
for n in names[:5]:
    weight.append(int(input(f"Introduce the weight of {n} (1-5): ")))

Introduce the weight of Old companies (1-5): 3
Introduce the weight of Starbucks (1-5): 5
Introduce the weight of Night bars (1-5): 4
Introduce the weight of Schools (1-5): 2
Introduce the weight of Startups (1-5): 3


In [5]:
points,adresses,air_dist = R.pointsFunction(collections,office_coords,office_info,air_coord,weight)

Creamos un nuevo Dataframe con la informacion a cerca de cada una de las oficinas donde podriamos potencialmente colocar la empresa:
    - Direccion
    - Tamaño de la oficina
    - Puntuacion en el ranking total
    - Distancia a startups, starbucks, compañias antiguas, aeropuerto, colegios y bares mas cercanos
    - Coordenadas
    - Referencia

In [6]:
url_1 = "https://42floors.com/"

df_offices = pd.DataFrame(columns=['Adress','Square Feet','Punctuation','Startups','Starbucks',
                                   'Old companies','Airport','Schools','Night bars','Location','Reference'])
 
for index in range(len(points)):
    total = sum(points[index][1:])-points[index][0]
    df_offices = df_offices.append({'Adress':adresses[index],'Punctuation':total,
                                    'Starbucks':points[index][1],'Schools':points[index][3],'Startups':points[index][4],
                                    'Old companies':-points[index][0],'Night bars':points[index][2],
                                    'Square Feet':of_size[index],'Location':office_coords[index],
                                   'Reference':url_1+of_ref[index],'Airport':air_dist[index]},ignore_index=True)

In [7]:
df = df_offices.sort_values(by=['Punctuation'],ascending=False)
df.to_csv('../output/office-location.csv')
df = df.reset_index(drop=True)
df

,Adress,Square Feet,Punctuation,Startups,Starbucks,Old companies,Airport,Schools,Night bars,Location,Reference
0,"255 State St, Boston, MA 02109, USA","19,940",25.50,6.75,12.50,-5.25,3.511837054493131 km,1.5,10.0,"[-71.0513553, 42.3593857]",https://42floors.com//us/ma/boston/255-state-s...
1,"5 Broad St, Boston, MA 02110, USA","11,050",21.00,6.75,12.50,-6.75,3.8058028687439767 km,1.5,7.0,"[-71.0548501, 42.3588513]",https://42floors.com//us/ma/boston/75-state-st...
2,"33 Broad St, Boston, MA 02109, USA","1,160",19.25,6.75,11.25,-6.75,3.786277661993289 km,1.0,7.0,"[-71.05447649999999, 42.3583857]",https://42floors.com//us/ma/boston/33-broad-st...
3,"84 State St, Boston, MA 02109, USA","10,450",19.25,6.00,12.50,-6.75,3.863713108254033 km,1.5,6.0,"[-71.0556458, 42.3591529]",https://42floors.com//us/ma/boston/84-state-st...
4,"33 Broad St, Boston, MA 02109, USA","3,560",19.25,6.75,11.25,-6.75,3.786277661993289 km,1.0,7.0,"[-71.05447649999999, 42.3583857]",https://42floors.com//us/ma/boston/33-broad-st...
...,...,...,...,...,...,...,...,...,...,...,...
63,"1 State St, New York, NY 10004, USA","2,040 - 4,600",0.00,0.00,0.00,-0.00,311.3277854543317 km,0.0,0.0,"[-74.0133207, 40.702703]",https://42floors.com//us/ma/boston/1-state-st?...
64,"207 South St, Elmhurst, IL 60126, USA","3,890",0.00,0.00,0.00,-0.00,1398.1919354859465 km,0.0,0.0,"[-87.93459089999999, 41.8857483]",https://42floors.com//us/ma/boston/207-south-s...
65,"70 Franklin St, New York, NY 10013, USA","10,050 - 11,430",0.00,0.00,0.00,-0.00,309.6816058391006 km,0.0,0.0,"[-74.004108, 40.7179659]",https://42floors.com//us/ma/boston/70-franklin...
66,"333 Newbury St #335, Springfield, MA 01104, USA","3,100 - 4,010",0.00,0.00,0.00,-0.00,132.18482077888294 km,0.0,0.0,"[-72.5785126, 42.1263255]",https://42floors.com//us/ma/boston/333-newbury...


In [8]:
coordinates = int(input("Introduce the index of the office you want:" ))
selected_office = df["Location"].iloc[coordinates][::-1]

Introduce the index of the office you want:0


In [9]:
M.mapResults(selected_office,16,names,colors,collections,air_coord,selected_office)